# Домашнее задание № 8

## Задание 1 (4 балла) 

Обучите 7 моделей для задачи классификации текста (датасет - lenta_40k ). А именно:  
1) модель с 1 GRU слоем;   
2) модель с 1 LSTM слоем    
3) модель с 1 GRU и 1 LSTM слоем  
4) модель с 1 BIGRU и 2 LSTM слоями  
5) модель с 5 GRU слоями и 3 LSTM слоями  
6) модель 1 BIGRU и 1 BILSTM слоями, причем так чтобы модели для forward и backward прохода отличались   
7) модель, где последовательно идут слои: LSTM, GRU, BILSTM, BIGRU, GRU, LSTM  



Параметр units и размер эмбединга можете задать любой. Оцените качество каждой модели и определите победителя.

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from string import punctuation
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


Считываем данные — новостные тексты с lenta.ru

In [ ]:
data = pd.read_csv('lenta_40k.csv')

Функция для простой предобработки

In [ ]:
def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens

Функция для рассчета f-меры в TensorFlow (источник: stackoverflow)

In [ ]:
from tensorflow.keras import backend as K
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Создаем словарь и фильтруем его, оставляя только частотные слова


In [ ]:
vocab = Counter()

for text in data.text:
    vocab.update(preprocess(text))

Оставим в словаре слова, которые встретились в корпусе не менее 30 раз

In [ ]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30: 
        filtered_vocab.add(word)

Создадим словари для перевода слов в индексы и наоборот

In [ ]:
word2id = {'PAD':0, 'UNK':1}

for word in filtered_vocab:
    word2id[word] = len(word2id)

In [ ]:
id2word = {i:word for word, i in word2id.items()}

In [ ]:
X = []

for text in data.text:
    tokens = preprocess(text)
    ids = [word2id.get(token, 1) for token in tokens]
    X.append(ids)

In [ ]:
# padding

MAX_LEN = max(len(x) for x in X)
MEAN_LEN = np.median([len(x) for x in X])
MAX_LEN = int(MEAN_LEN + 30)
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_LEN)

In [ ]:
X.shape

(44356, 200)

In [ ]:
id2label = {i:label for i, label in enumerate(set(data.topic.values))}
label2id = {l:i for i, l in id2label.items()}

In [ ]:
y = tf.keras.utils.to_categorical([label2id[label] for label in data.topic.values])

Добавим стратификацию, т.к. в данных дисбаланс классов

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05, stratify=y)

## Models

In [ ]:
scores = {} 

#### Model 1: модель с 1 GRU слоем


In [ ]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )

gru_layer = tf.keras.layers.GRU(128, return_sequences=False)(embeddings)

outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(gru_layer)

model_1 = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_1.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1, tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

In [ ]:
model_1.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000,
          epochs=20)

In [ ]:
model_1.history.history['f1'][-1]

0.946437418460846

In [ ]:
scores['1 GRU слой'] = model_1.history.history['f1'][-1]

#### Model 2: модель с 1 LSTM слоем


In [ ]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )

lstm_layer = tf.keras.layers.LSTM(128, return_sequences=False)(embeddings)

outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(lstm_layer)

model_2 = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_2.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1, tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

In [ ]:
model_2.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000,
          epochs=20)

In [ ]:
model_2.history.history['f1'][-1]

0.9168779850006104

In [ ]:
scores['1 LSTM слой'] = model_2.history.history['f1'][-1]

#### Model 3: модель с 1 GRU и 1 LSTM слоем

In [ ]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )

lstm_1 = tf.keras.layers.LSTM(128, return_sequences=True)(embeddings)
lstm_2 = tf.keras.layers.GRU(128, return_sequences=False)(lstm_1)

dense = tf.keras.layers.Dense(64, activation='relu')(lstm_2)
outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(dense)

model_3 = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_3.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1, tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])


In [ ]:
model_3.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000,
          epochs=20)

In [ ]:
model_3.history.history['f1'][-1]

0.9580317735671997

In [ ]:
scores['1 GRU и 1 LSTM слой'] = model_3.history.history['f1'][-1]

#### Model 4: модель с 1 BiGRU и 2 LSTM слоями

In [ ]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )

bigru_layer = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(embeddings)
lstm_layer_1 = tf.keras.layers.LSTM(128, return_sequences=True)(bigru_layer)
lstm_layer_2 = tf.keras.layers.LSTM(128, return_sequences=False)(lstm_layer_1)

dense = tf.keras.layers.Dense(64, activation='relu')(lstm_layer_2)
outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(dense)

model_4 = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_4.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1, tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

In [ ]:
model_4.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000,
          epochs=20)

In [ ]:
model_4.history.history['f1'][-1]

0.9138382077217102

In [ ]:
scores['1 BiGRU и 2 LSTM слоя'] = model_4.history.history['f1'][-1]

#### Model 5: модель с 5 GRU и 3 LSTM слоями

In [ ]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )

# 5 GRU слоев
gru_1 = tf.keras.layers.GRU(128, return_sequences=True)(embeddings)
gru_2 = tf.keras.layers.GRU(128, return_sequences=True)(gru_1)
gru_3 = tf.keras.layers.GRU(128, return_sequences=True)(gru_2)
gru_4 = tf.keras.layers.GRU(128, return_sequences=True)(gru_3)
gru_5 = tf.keras.layers.GRU(128, return_sequences=True)(gru_4)

# 3 LSTM слоя
lstm_1 = tf.keras.layers.LSTM(128, return_sequences=True)(gru_5)
lstm_2 = tf.keras.layers.LSTM(128, return_sequences=True)(lstm_1)
lstm_3 = tf.keras.layers.LSTM(128, return_sequences=False)(lstm_2)

dense = tf.keras.layers.Dense(64, activation='relu')(lstm_3)
outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(dense)

model_5 = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_5.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1, tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

In [ ]:
model_5.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000,
          epochs=20)

In [ ]:
model_5.history.history['f1'][-1]

0.6030071973800659

In [ ]:
scores['5 GRU и 3 LSTM слоя'] = model_5.history.history['f1'][-1]

#### Model 6: модель с 1 BIGRU и 1 BILSTM слоями (причем так, чтобы модели для forward и backward прохода отличались)

In [ ]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )

bigru_bilstm = tf.keras.layers.Bidirectional(
                                       tf.keras.layers.GRU(128, return_sequences=False),
                        backward_layer=tf.keras.layers.LSTM(128, return_sequences=False, 
                                                            go_backwards=True))(embeddings)

outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(bigru_bilstm)
model_6 = tf.keras.Model(inputs=inputs, outputs=outputs)

model_6.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=[f1, tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

In [ ]:
model_6.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000,
          epochs=20)

In [ ]:
model_6.history.history['f1'][-1]

0.7875635623931885

In [ ]:
scores['1 BiGRU и 1 BiLSTM'] = model_6.history.history['f1'][-1]

#### Model 7. Модель с последовательностью слоев: LSTM, GRU, BiLSTM, BiGRU, GRU, LSTM

In [ ]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )

lstm_1 = tf.keras.layers.LSTM(128, return_sequences=True)(embeddings)
gru_1 = tf.keras.layers.GRU(128, return_sequences=True)(lstm_1)
bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(gru_1)
bigru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(bilstm)
gru_2 = tf.keras.layers.GRU(128, return_sequences=True)(bigru)
lstm_2_final = tf.keras.layers.LSTM(128, return_sequences=False)(gru_2)

dense = tf.keras.layers.Dense(64, activation='relu')(lstm_2_final)
outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(dense)

model_7 = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model_7.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1, tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

In [ ]:
model_7.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000,
          epochs=20)

In [ ]:
model_7.history.history['f1'][-1]

0.8944259285926819

In [ ]:
scores['LSTM, GRU, BiLSTM, BiGRU, GRU, LSTM'] = model_7.history.history['f1'][-1]

#### Оценка качества моделей, выбор лучшей модели

In [ ]:
scores_df = pd.DataFrame.from_dict(scores, orient='index', columns=['f-score'])

In [ ]:
scores_df.sort_values(by=['f-score'], ascending=False)

,f-score
1 GRU и 1 LSTM слой,0.958032
1 GRU слой,0.946437
1 LSTM слой,0.916878
1 BiGRU и 2 LSTM слоя,0.913838
"LSTM, GRU, BiLSTM, BiGRU, GRU, LSTM",0.894426
1 BiGRU и 1 BiLSTM,0.787564
5 GRU и 3 LSTM слоя,0.603007


В итоге, лучший результат в данной серии экспериментов был получен моделью с 1 GRU и 1 LSTM слоями. Не менее хорошие результаты показали модели, где использовалось по одному RNN слою: на втором месте модель с 1 GRU слоем, на третьем — модель с 1 LSTM слоем. Усложнение архитектуры модели в данном эксперименте не привело к улучшению качества результатов. Худшей оказалась модель с 5 GRU и 3 LSTM слоями.

## Задание 2 (6 баллов)


На данных википедии (wikiann) обучите 2 модели:  
1) модель в которой будут использованы предобученные эмбединги слов и несколько BILSTM слоев. 
1) модель в которой будут использованы предобученные эмбединги слов и несколько BIGRU слоев. 

Сравните качество по метрикам. Также придумайте несколько сложных примеров и проверьте, какие сущности определяет каждая из моделей.

Загрузим датасет wikiann


In [ ]:
!pip install datasets
from datasets import load_dataset

In [ ]:
wikiann_dataset = load_dataset("wikiann", 'ru')

Создадим словарь и проиндексируем его

In [5]:
vocab = Counter()

for sent in wikiann_dataset['train']['tokens']:
    vocab.update([x.lower() for x in sent])

In [6]:
word2id = {'PAD':0, 'UNK':1}

for word in vocab:
    word2id[word] = len(word2id)

In [7]:
id2word = {i:word for word, i in word2id.items()}

In [8]:
X = []

for sent in wikiann_dataset['train']['tokens']:
    tokens = [w.lower() for w in sent]
    ids = [word2id.get(token, 1) for token in tokens]
    X.append(ids)

In [9]:
X_test = [] # тесты

for sent in wikiann_dataset['test']['tokens']:
    tokens = [w.lower() for w in sent]
    ids = [word2id.get(token, 1) for token in tokens]
    X_test.append(ids)

In [10]:
MAX_LEN = max(len(x) for x in X)

# padding
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_LEN, padding='post')
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=MAX_LEN, padding='post')

In [11]:
# обратный маппинг
id2labels = {0:'O', 1:'B-PER', 2:'I-PER', 3:'B-ORG', 4:'I-ORG', 5: 'B-LOC', 6:'I-LOC', 7:'PAD'}
label2id = {v:k for k,v in id2labels.items()} 

In [12]:
y = tf.keras.preprocessing.sequence.pad_sequences(wikiann_dataset['train']['ner_tags'], value=7,
                                                  maxlen=MAX_LEN,  padding='post')
y_test = tf.keras.preprocessing.sequence.pad_sequences(wikiann_dataset['test']['ner_tags'], value=7,
                                                       maxlen=MAX_LEN,  padding='post')

#### Модель с 3-мя BiLSTM слоями



In [13]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=100)(inputs)

lstm_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(embeddings)
lstm_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(lstm_1)
lstm_3 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(lstm_2)

outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(lstm_3)

bilstm_model = tf.keras.Model(inputs=inputs, outputs=outputs)
bilstm_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', 
             metrics=['accuracy'])

In [14]:
bilstm_model.fit(X, y, 
          validation_data=(X_test, y_test),
          batch_size=128,
         epochs=5)

Epoch 1/5
157/157 [==============================] - 38s 153ms/step - loss: 0.2424 - accuracy: 0.9318 - val_loss: 0.1502 - val_accuracy: 0.9523
Epoch 2/5
157/157 [==============================] - 18s 115ms/step - loss: 0.1267 - accuracy: 0.9595 - val_loss: 0.1079 - val_accuracy: 0.9649
Epoch 3/5
157/157 [==============================] - 18s 115ms/step - loss: 0.0763 - accuracy: 0.9757 - val_loss: 0.0662 - val_accuracy: 0.9791
Epoch 4/5
157/157 [==============================] - 18s 115ms/step - loss: 0.0322 - accuracy: 0.9899 - val_loss: 0.0582 - val_accuracy: 0.9824
Epoch 5/5
157/157 [==============================] - 18s 116ms/step - loss: 0.0147 - accuracy: 0.9956 - val_loss: 0.0652 - val_accuracy: 0.9810


In [15]:
pred = bilstm_model.predict(X_test).argmax(2)

In [16]:
print(classification_report(y_test.reshape(-1), pred.reshape(-1), labels=list(id2labels.keys()),
                                                                     target_names=list(id2labels.values()),
                                                                     zero_division=0))

              precision    recall  f1-score   support

           O       0.94      0.93      0.94     40480
       B-PER       0.95      0.75      0.84      3542
       I-PER       0.95      0.73      0.83      7544
       B-ORG       0.71      0.69      0.70      4074
       I-ORG       0.83      0.78      0.80      8008
       B-LOC       0.60      0.80      0.68      4560
       I-LOC       0.48      0.79      0.60      3060
         PAD       1.00      1.00      1.00    468732

    accuracy                           0.98    540000
   macro avg       0.81      0.81      0.80    540000
weighted avg       0.98      0.98      0.98    540000



#### Модель с 3-мя BiGRU слоями

In [17]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=100)(inputs)

gru_1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(embeddings)
gru_2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(gru_1)
gru_3 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(gru_2)

outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(gru_3)

bigru_model = tf.keras.Model(inputs=inputs, outputs=outputs)
bigru_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', 
             metrics=['accuracy'])

In [18]:
bigru_model.fit(X, y, 
          validation_data=(X_test, y_test),
          batch_size=128,
          epochs=5)

Epoch 1/5
157/157 [==============================] - 30s 125ms/step - loss: 0.2293 - accuracy: 0.9344 - val_loss: 0.1051 - val_accuracy: 0.9661
Epoch 2/5
157/157 [==============================] - 17s 106ms/step - loss: 0.0601 - accuracy: 0.9809 - val_loss: 0.0497 - val_accuracy: 0.9843
Epoch 3/5
157/157 [==============================] - 17s 109ms/step - loss: 0.0190 - accuracy: 0.9942 - val_loss: 0.0497 - val_accuracy: 0.9853
Epoch 4/5
157/157 [==============================] - 17s 110ms/step - loss: 0.0089 - accuracy: 0.9974 - val_loss: 0.0546 - val_accuracy: 0.9854
Epoch 5/5
157/157 [==============================] - 17s 106ms/step - loss: 0.0054 - accuracy: 0.9985 - val_loss: 0.0596 - val_accuracy: 0.9855


In [19]:
pred = bigru_model.predict(X_test).argmax(2)

In [20]:
print(classification_report(y_test.reshape(-1), pred.reshape(-1), labels=list(id2labels.keys()),
                                                                     target_names=list(id2labels.values()),
                                                                     zero_division=0))

              precision    recall  f1-score   support

           O       0.94      0.95      0.95     40480
       B-PER       0.88      0.87      0.88      3542
       I-PER       0.91      0.90      0.90      7544
       B-ORG       0.69      0.73      0.71      4074
       I-ORG       0.85      0.79      0.82      8008
       B-LOC       0.70      0.80      0.75      4560
       I-LOC       0.90      0.69      0.78      3060
         PAD       1.00      1.00      1.00    468732

    accuracy                           0.99    540000
   macro avg       0.86      0.84      0.85    540000
weighted avg       0.99      0.99      0.99    540000



#### Анализ результатов, проверка модели на своих примерах

Если сравнивать модели по метрикам, то их качество практически не отличается: значение f-меры для модели на основе BiLSTM равно 0.98, а для модели BiGRU — 0.99. 
<br>
Наименьшее значение f-меры для BiLSTM модели соответствует классу I-LOC (f1 = 0.60). Модель BiGRU хуже всего предсказывает класс B-ORG (f1 = 0.71).

Проверим работу моделей на нескольких реальных примерах:

In [21]:
import re

def tokenize(text, word2id):
    # токенизирует и переводит в индексы
    tokens = re.findall('\w+|[^\w\s]+', text)
    ids = [word2id.get(token.lower(), 1) for token in tokens]
    return tokens, ids

def pred2tags(pred, id2label, length):
    # декодирует индексы в части речи
    # length нужно чтобы откидывать паддинги или некорректные предсказания
    pred = pred.argmax(2)[0, :length]
    labels = [id2label[l] for l in pred]
    return labels

def label_seq(text, word2id, id2label, max_len, model):
    tokens, ids = tokenize(text, word2id)
    pred = model.predict(tf.keras.preprocessing.sequence.pad_sequences([ids], 
                                                                       maxlen=max_len, 
                                                                       padding='post'))
    labels = pred2tags(pred, id2label, len(ids))
    
    return list(zip(tokens, labels))

In [46]:
label_seq('Маша ездила навестить своих родственников в Татарстан.', word2id, id2labels, MAX_LEN, bilstm_model)

[('Маша', 'B-ORG'),
 ('ездила', 'I-ORG'),
 ('навестить', 'I-ORG'),
 ('своих', 'O'),
 ('родственников', 'O'),
 ('в', 'O'),
 ('Татарстан', 'B-LOC'),
 ('.', 'O')]

In [47]:
label_seq('Маша ездила навестить своих родственников в Татарстан.', word2id, id2labels, MAX_LEN, bigru_model)

[('Маша', 'B-PER'),
 ('ездила', 'I-PER'),
 ('навестить', 'O'),
 ('своих', 'O'),
 ('родственников', 'O'),
 ('в', 'O'),
 ('Татарстан', 'B-LOC'),
 ('.', 'O')]

Видно, что даже на достаточно простых примерах модели могут делать грубые ошибки. Так, модель на основе BiLSTM неверно присвоила именованной сущности "Маша" тег B-ORG. При этом модель на основе BiGRU присвоила слову верный тег (B-PER). 

In [48]:
label_seq('В городе Симферополь закрылся аэропорт.', word2id, id2labels, MAX_LEN, bilstm_model)

[('В', 'O'),
 ('городе', 'O'),
 ('Симферополь', 'B-ORG'),
 ('закрылся', 'I-ORG'),
 ('аэропорт', 'I-ORG'),
 ('.', 'O')]

In [49]:
label_seq('В городе Симферополь закрылся аэропорт.', word2id, id2labels, MAX_LEN, bigru_model)

[('В', 'O'),
 ('городе', 'O'),
 ('Симферополь', 'B-LOC'),
 ('закрылся', 'I-ORG'),
 ('аэропорт', 'I-ORG'),
 ('.', 'O')]

В примере выше модель на основе BiLSTM снова уступила BiGRU, присвоив неверный тег именованной сущности "Симферополь" (ORG вместо LOC).